In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data = pd.read_csv('/kaggle/input/kaggle-survey-2020/kaggle_survey_2020_responses.csv')


# Hello and Welcome to my Notebook

On holidays for a little while and figured I might as well try to do some analysis... Hope y'all enjoy! 

   #### *Setting the scene*

Come along for the journey at the end of 2020 to discover the pot of gold at the end of the rainbow, the unicorn of all unicorns, or simply just 
#### *THE ultimate skills to have in the ML and data science field. * 

### Buckle up and I hope you enjoy the ride! 



Steps Taken:

1. [The Start](#start)
<!--- 2. [Spice it up](#spicy) 
3.[Not making the cut so far... work in progress](#cut) */ -->



To answer this question, there were a couple of ways to 

<a id="start">

<a id="start"> Brainstorming Session </a>

This task seemed daunting due to the sheer amount of what could be defined  as the ultimate data sciene and ML skills. 

For me, it means what skills are used today. ie languages, data viz, BI tools, and to wrap it all up, what are they planning to learn in the future. 



## Starting Point

I wanted to know more about the respondents of the Kaggle survey. From the intro page, I found out that there were 20,036 responses to work with. 

#### Step 1: 
- Who responded to the survey?  
- What are the respondents coding experience?
- What are the respondents' occupations?

In [ ]:
data = pd.read_csv('/kaggle/input/kaggle-survey-2020/kaggle_survey_2020_responses.csv')

## Step 1: Determine how many individuals can be included in the question... 
coding_experience = pd.DataFrame({'count' : data[1:].groupby( [ 'Q6'] ).size()}).reset_index()

plt.figure(figsize = (6,3))
sns.barplot(x='Q6', y='count',data=coding_experience.sort_values(by=['count'], ascending=False), palette="Blues",  linewidth=1, edgecolor='k')
plt.title("Coder vs. Non-Coder \n \n Q6.For how many years have you been writing code and/or programming? \n",fontsize=16)
plt.xlabel('Coding Experience',fontsize=12)
plt.ylabel('# of Occurences')
plt.xticks(rotation=45 , ha='right')
plt.show()

occupation = pd.DataFrame({'count' : data[1:].groupby( [ 'Q5'] ).size()}).reset_index()
plt.figure(figsize = (6,3))
sns.barplot(x='Q5', y='count',data=occupation.sort_values(by=['count'], ascending=False), palette="Greys_r",  linewidth=1, edgecolor='k')
plt.title("Occupation\n  \n Q5.Select the title most similar to your current role (or most recent title if retired)\n",fontsize=16)
plt.xlabel('Occupation',fontsize=12)
plt.ylabel('# of Occurences')
plt.xticks(rotation=45 , ha='right')
plt.show()


Narrowing down the field to my target respondents:
 - non-students
 - employed  
 - respondents who have written a line of code. 
 
 Below is the updated visuals. 

In [ ]:
occupation_coders = data[(data['Q6'] != 'I have never written code') & 
                         (data['Q5'] != 'Student') & 
                         (data['Q5'] != 'Currently not employed')][['Q5','Q6']][1:]

occupation_coders_experience = pd.DataFrame({'count' : occupation_coders.groupby( [ 'Q6'] ).size()}).reset_index()
plt.figure(figsize = (6,3))
sns.barplot(x='Q6', y='count',data=occupation_coders_experience.sort_values(by=['count'], ascending=False), palette="Blues",  linewidth=1, edgecolor='k')
plt.title("Occupation of Coders\n  \n For how many years have you been writing code and/or programming?\n",fontsize=16)
plt.xlabel('Years of Experience',fontsize=12)
plt.ylabel('# of Occurences')
plt.xticks(rotation=45 , ha='right')
plt.show()


occupation_coders_count = pd.DataFrame({'count' : occupation_coders.groupby( [ 'Q5'] ).size()}).reset_index()
plt.figure(figsize = (6,3))
sns.barplot(x='Q5', y='count',data=occupation_coders_count.sort_values(by=['count'], ascending=False), palette="Greys_r",  linewidth=1, edgecolor='k')
plt.title("Occupation of Coders\n  \n Q5.Select the title most similar to your current role (or most recent title if retired)\n",fontsize=16)
plt.xlabel('Occupation',fontsize=12)
plt.ylabel('# of Occurences')
plt.xticks(rotation=45 , ha='right')
plt.show()


To continue my analysis, I want to determine the years of experience by occupation and then break it down every further. 


In [ ]:
subset = data[(data['Q6'] != 'I have never written code') & 
              (data['Q5'] != 'Student') & 
              (data['Q5'] != 'Currently not employed')][['Q1','Q5','Q6']][1:]


#creating a pivot table for the heatmap (age by num of languages)
pivot = pd.pivot_table(subset, index=['Q5'], columns= ['Q6'], aggfunc='count',fill_value =0)
cols = list(pivot.columns.get_level_values(level=1))
index = list(pivot.index.get_level_values(level=0))

output = pd.DataFrame(pivot.values, index=index, columns=cols)

output = output.reindex(columns=['< 1 years', '1-2 years', '3-5 years', '5-10 years','10-20 years', '20+ years'])

## Occupation by Years of Experience

plt.figure(figsize = (10,6))
sns.heatmap(output,cmap="Reds", linewidths=0.5, annot=False, annot_kws={"fontsize":8})
plt.yticks(rotation=0)
plt.title('Occupation by Years of Coding Experience\n \n By # of responses',fontsize=16)
plt.ylabel('Occupation')
plt.xlabel('Years of Experience')
plt.show()

In [ ]:
## need to find the percentage by row 

output['responses'] = list(output.sum(axis=1))
#output.head()

for col in output.columns:
    if col != "responses":
        col_name = "perc_" + col
        output[col_name] = (output[col] / output['responses']) * 100



### Finding the % of Coding Experience By Occupation

In [ ]:
category_names = ['< 1 years', '1-2 years', '3-5 years', '5-10 years','10-20 years', '20+ years']

results = {
    'Business Analyst': list(output.iloc[0,7:13].values),
    'DBA/Database Engineer': list(output.iloc[1,7:13].values),
    'Data Analyst': list(output.iloc[2,7:13].values),
    'Data Engineer': list(output.iloc[3,7:13].values),
    'Data Scientist': list(output.iloc[4,7:13].values),
    'Machine Learning Engineer': list(output.iloc[5,7:13].values),
    'Other':list(output.iloc[6,7:13].values),
    'Product/Project Manager':list(output.iloc[7,7:13].values),
    'Research Scientist':list(output.iloc[8,7:13].values),
    'Software Engineer':list(output.iloc[9,7:13].values),
    'Statistician': list(output.iloc[10,7:13].values)
    
}
## REFERENCE: https://matplotlib.org/3.1.1/gallery/lines_bars_and_markers/horizontal_barchart_distribution.html#sphx-glr-gallery-lines-bars-and-markers-horizontal-barchart-distribution-py
def survey(results, category_names):
    """
    Parameters
    ----------
    results : dict
        A mapping from question labels to a list of answers per category.
        It is assumed all lists contain the same number of entries and that
        it matches the length of *category_names*.
    category_names : list of str
        The category labels.
    """
    labels = list(results.keys())
    data = np.array(list(results.values()))
    data_cum = data.cumsum(axis=1)
    category_colors = plt.get_cmap('RdYlGn')(
        np.linspace(0.15, 0.85, data.shape[1]))

    fig, ax = plt.subplots(figsize=(9.2, 5))
    ax.invert_yaxis()
    ax.xaxis.set_visible(False)
    ax.set_xlim(0, np.sum(data, axis=1).max())

    for i, (colname, color) in enumerate(zip(category_names, category_colors)):
        widths = data[:, i]
        starts = data_cum[:, i] - widths
        ax.barh(labels, widths, left=starts, height=0.5,
                label=colname, color=color)
        xcenters = starts + widths / 2

        r, g, b, _ = color
        text_color = 'white' if r * g * b < 0.5 else 'darkgrey'
        for y, (x, c) in enumerate(zip(xcenters, widths)):
            ax.text(x, y, str(int(c)), ha='center', va='center',
                    color=text_color)
    ax.legend(ncol=len(category_names), bbox_to_anchor=(0, 1),
              loc='lower left', fontsize='small')

    return fig, ax

#plt.title("% of Years of Experience By Occupation")

survey(results, category_names)
plt.show()

# To Be Continued.... 

# Feel free to leave comments and feedback :) 